# Knapsack: Greedy and 2-Approximation

In the following, we will use the knapsack instance from the lecture as an easy example with a weight capacity of $W=10$. We will index the items from $1$ to $7$.

| item   |   1 |  2 |   3 |   4 |  5 |   6 |  7  |
|--------|-----|----|-----|-----|-----|----|-----|
| weight |   4 |  1 |   2 |   3 |   2 |  1 |   2 |
| value  | 299 | 73 | 159 | 221 | 137 | 89 | 157 |

In [1]:
class Item:
    def __init__(self, number, weight=0, value=0):
        self.number = number
        self.value = value
        self.weight = weight
        
items = [Item(1,4,299), Item(2,1,73), Item(3,2,159), Item(4,3,221), Item(5,2,137), Item(6,1,89), Item(7,2,157)]
capacity = 10

We will start by implementing the simple greedy approach by sorting for the values only. Try to fill in the blanks in the following code to get a working implementation of that algorithm. We use `sorted` to sort our list of items. The `key` argument is a function that returns some comparable value that is used as the basis for sorting the list. Here, we use a so-called _lambda function_ to declare a small inline function that take one argument `x` and returns `x.value`. If we feed an `Item` object to that function, it will returns its value, so we effectively sort the list by values. As the normal sorting order is ascending, we also need to supply the `reverse=True` parameter. 

In [2]:
sorted_items = sorted(items, key = lambda x: x.value, reverse=True)

In [3]:
for item in sorted_items:
    print(f"item {item.number} has value {item.value} and weight {item.weight}")

item 1 has value 299 and weight 4
item 4 has value 221 and weight 3
item 3 has value 159 and weight 2
item 7 has value 157 and weight 2
item 5 has value 137 and weight 2
item 6 has value 89 and weight 1
item 2 has value 73 and weight 1


In [4]:
selected_items = set()
total_value = 0
total_weight = 0
i = 0
while total_weight + sorted_items[i].weight <= capacity:
    selected_items.add(sorted_items[i].number)
    total_weight += sorted_items[i].weight
    total_value += sorted_items[i].value
    i += 1

print(f"Computed solution has a total weight of {total_weight}, a total value of {total_value} and contains the items {selected_items}.")

Computed solution has a total weight of 9, a total value of 679 and contains the items {1, 3, 4}.


## Problem 1
Try to modify the algorithm above so that it yields the 2-approximation algorithm discussed in the lecture.

We start by modifying the sorting: Instead of by value, we now sort by the ration of value to weight. Note that in the output we use the _type specifier_ `:.2f` to print the ratio as a floating point number with 2 decimals.

In [8]:
sorted_items = sorted(items, key = lambda x: x.value/x.weight, reverse=True)

In [9]:
for item in sorted_items:
    print(f"item {item.number} has value {item.value} and weight {item.weight}, the relative value is {item.value/item.weight:.2f}")

item 6 has value 89 and weight 1, the relative value is 89.00
item 3 has value 159 and weight 2, the relative value is 79.50
item 7 has value 157 and weight 2, the relative value is 78.50
item 1 has value 299 and weight 4, the relative value is 74.75
item 4 has value 221 and weight 3, the relative value is 73.67
item 2 has value 73 and weight 1, the relative value is 73.00
item 5 has value 137 and weight 2, the relative value is 68.50


The modified algorithm is then straightforward: We use the original and just add an `if` clause at the end to test whether our solution is better than just taking the next item.

In [11]:
selected_items = set()
total_value = 0
total_weight = 0
i = 0
while total_weight + sorted_items[i].weight <= capacity:
    selected_items.add(sorted_items[i].number)
    total_weight += sorted_items[i].weight
    total_value += sorted_items[i].value
    i += 1
    
if total_value < sorted_items[i].value:
    selected_items = {i}
    total_value = sorted_items[i].value
    total_weight = sorted_items[i].weight

print(f"Computed solution has a total weight of {total_weight}, a total value of {total_value} and contains the items {selected_items}.")

Computed solution has a total weight of 9, a total value of 704 and contains the items {1, 3, 6, 7}.


## Problem 2: Dynamic Programming for Knapsack

For an exact solution of the knapsack problem, we have discussed a dynamic programming approach. Define $C_j(\gamma)$ as the minimum total weight of a feasible knapsack on items in $\{0,\ldots, j\}$ with exactly value $\gamma$. Then the following recursion can be used:
\begin{align*}
   C_j(\gamma) &= \min\left\{C_{j-1}(\gamma), C_{j-1}(\gamma-v_j) + w_j\right\}\\
   C_{j}(\gamma) &= \infty, \quad \text{if it exceeds $W$}
\end{align*}
where we set initial values as
\begin{align*}
    C_0(\gamma) &=
                  \begin{cases}
                    0, &\text{for $\gamma = 0$},\\
                    \infty, &\text{otherwise}.
                  \end{cases}
  \end{align*}
  
1. Implement this dynamic programming algorithm and test it on the knapsack instance defined above.
1. Make sure your algorithm does not just report the value of the solution, but also the corresponding knapsack.

* In the following code we use a list `C` to hold the values of $C_j(\gamma)$, where each list entry corresponds to one value of $j$ (i.e. `C[0]` corresponds to $j=0$, etc.). Each entry of this list is a dictionary where the keys are the admissible values for $\gamma$ and the values are the corresponding $C_j(\gamma)$. We do not store infinity values but only store $C_j(\gamma)$ for $\gamma$ where the value is finite. 
* The list `S` has a structure identical to that of `C`. Again, each entry of the list corresponds to a value of $j$ and contains a dictionary where the keys are possible values of $\gamma$. The values stored there are the knapsack solutions corresponding to the respective `C` values.
* Note that while we do store all values of $C$ completely, that would not be necessary. In fact, it is sufficient to store the values of $C_{j-1}$ for building $C_j$, so we could throw away all previous values. For the code, that means we would only need a list with 2 entries, or just two dictionaries `C_old` and `C_new`. We chose to implement the algorithm differently because that enables us to inspect all values at the end which is helpful for debugging and for understanding the algorithm. You would probably opt for the more memory-efficient solution in production code (especially for large instances).
* Note that an empty set in Python is initialized with `set()`, while `{}` creates an empty dictionary.

In [12]:
C = [] # this will hold the values of C_j; each entry is 
S = []
C.append({0: 0})
S.append({0: set()})
for j in range(1,len(items)+1):
    #copy solutions for j-1 first, all these are also feasible for j
    new_layer = {gamma: C[j-1][gamma] for gamma in C[j-1]} # next entry of C
    new_layer_solutions = {gamma: S[j-1][gamma] for gamma in C[j-1]} # next entry of S
    # compute weights that may be reached with item j based on all weights for items in [j-1]
    for gamma in C[j-1]: #iterate over keys = gamma-values
        new_value = gamma + items[j-1].value # remember: items are index from 0, but loop starts at 1!
        new_weight = C[j-1][gamma] + items[j-1].weight
        new_solution = set(S[j-1][gamma])
        new_solution.add(j)
        if new_weight <= capacity: # new solution feasible?
            if (new_value in new_layer): #is gamma value already present in new layer?
                if new_layer[new_value] > new_weight: # replace if new solution is better
                    new_layer[new_value] = new_weight
                    new_layer_solutions[new_value] = new_solution
            else: # create new entry for gamma value
                new_layer[new_value] = new_weight
                new_layer_solutions[new_value] = new_solution
    C.append(new_layer) # store layer ...
    S.append(new_layer_solutions) # ... and solution
        
max_value = max(C[len(items)]) # find best value in last layer (max over keys of the dictionary!)
print(f"Computed solution has a total weight of {C[len(items)][max_value]} and a total value of {max_value}.")
print(f"The solution is {S[len(items)][max_value]}.")
        

Computed solution has a total weight of 10 and a total value of 777.
The solution is {1, 2, 3, 6, 7}.


## Problem 3: An FPTAS for Knapsack
Modify your solution to implement the FPTAS from the lecture.

_This problem is left as an exercise for the reader, as it is a straightforward modification of the above dynamic programming algorithm._